In [0]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.layers.core import Layer
from keras.initializers import RandomNormal
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import RMSprop, SGD
from keras.utils import to_categorical,plot_model

In [2]:
image_width=32
image_height=32
image_depth=3
def load_cifar10(num_training=47000, num_validation=3000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel)
    X_val = (X_val - mean_pixel)
    X_test = (X_test - mean_pixel)

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()       
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape) 

170500096/170498071 [==============================] - 5s 0us/step
Train data shape:  (47000, 32, 32, 3)
Train labels shape:  (47000, 10) float32
Validation data shape:  (3000, 32, 32, 3)
Validation labels shape:  (3000, 10)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000, 10)


In [0]:
batchSize = 256
epoch = 5
optimizer = SGD(lr=0.01, momentum=0.9, decay=0.0005, nesterov=True)
numClasses = 10

In [0]:
Initializer1 = RandomNormal(mean=0.0, stddev=0.01, seed=None)
model = Sequential([
    Conv2D(96,(11,11),strides = (4,4), activation = "relu", use_bias = True, bias_initializer='zeros',padding="same",kernel_initializer=Initializer1,input_shape=(image_height,image_width,image_depth,)),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    BatchNormalization(),
    Conv2D(256,(5,5),strides =(1,1),kernel_initializer=Initializer1, activation = "relu",padding="same", use_bias = True, bias_initializer='ones'),
    MaxPooling2D(pool_size=(3, 3), strides=(2,2)),
    BatchNormalization(),
    Conv2D(384,(3,3),strides =(1,1),kernel_initializer=Initializer1,padding="same", use_bias = True, bias_initializer='zeros'),
    Conv2D(384,(3,3),strides =(1,1),kernel_initializer=Initializer1,padding="same", use_bias = True, bias_initializer='ones'),
    Conv2D(256,(3,3),strides =(1,1),kernel_initializer=Initializer1,padding="same", use_bias = True, bias_initializer='ones'),
    MaxPooling2D(pool_size=(3, 3), strides=(2,2),padding="same"),
    BatchNormalization(),
    Flatten(),
    Dense(4096, activation='relu',kernel_initializer=Initializer1, use_bias = True, bias_initializer='zeros'),
    Dropout(0.5),
    Dense(4096, activation='relu',kernel_initializer=Initializer1, use_bias = True, bias_initializer='zeros'),
    Dropout(0.5),
    Dense(10,kernel_initializer=Initializer1, use_bias = True, bias_initializer='zeros', activation='softmax'),
])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 8, 8, 96)          34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 96)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 96)          384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 256)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 1, 256)         1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 1, 384)         885120    
__________

In [0]:
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

In [12]:
model.fit(X_train, y_train,batch_size=batchSize,epochs=25,verbose=1,validation_data=(X_val, y_val))

Train on 47000 samples, validate on 3000 samples
Epoch 1/25
16896/47000 [=========>....................] - ETA: 9s - loss: 0.0034 - acc: 0.9995 - top_k_categorical_accuracy: 1.0000

KeyboardInterrupt: ignored

In [15]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test Top 5 accuracy:', score[2])

10000/10000 [==============================] - 3s 260us/step
Test loss: 2.527464137649536
Test accuracy: 0.6397
Test Top 5 accuracy: 0.956
